In [58]:
import pandas as pd
from datetime import datetime

In [59]:
resigned = pd.read_csv("resigned.csv")
reasons = pd.read_csv("reasons.csv")

Drop the duplicate Staff IDs:

In [60]:
resigned.drop_duplicates(subset='Staff ID', keep='last', inplace=True)
reasons.drop_duplicates(subset='Number', keep='last', inplace=True)

Data cleaning:

In [61]:
#Resigned data:
resigned['Date of Leaving'] = pd.to_datetime(resigned['Date of Leaving'], format='%d/%m/%y') + pd.offsets.MonthEnd(n=1)
resigned['Staff ID'] = resigned['Staff ID'].astype(int)

In [62]:
#Reasons data:
reasons.drop(['Timestamp'], axis=1, inplace=True)
reasons.replace("Yes", 1, inplace=True)
reasons['Difficulty with location/transport'] = reasons[["Difficulty with location", "Transport problems"]].max(axis=1)
reasons['Cannot adapt to working environment/working conditions'] = reasons[["Cannot adapt to working environement", "Working conditions"]].max(axis=1)
reasons['Health issues'] = reasons[["Health issues unrelated to work", "Health issues related to work"]].max(axis=1)
reasons['Lack of career development/promotion prospects'] = reasons[["Lack of career development", "No promotional prospects"]].max(axis=1)
reasons.drop(['Difficulty with location', 'Transport problems'], axis=1, inplace=True)
reasons.drop(['Cannot adapt to working environement', 'Working conditions'], axis=1, inplace=True)
reasons.drop(['Health issues unrelated to work', 'Health issues related to work'], axis=1, inplace=True)
reasons.drop(['Lack of career development', 'No promotional prospects'], axis=1, inplace=True)
reasons.rename(columns={'Difficulty with clients': 'Difficulties with clients', 'Difficulty with colleagues': 
                        'Difficulties with colleagues', 'Difficulty with supervisors': 'Difficulties with supervisors',
                       'Number': 'Staff ID'}, inplace=True)
reasons['Staff ID'].fillna(0, inplace=True)
reasons['Staff ID'] = reasons['Staff ID'].astype(int)
reasons_melted = pd.melt(reasons, id_vars=['Staff ID'])
reasons_melted.dropna(inplace=True)

In [63]:
reasons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2508 entries, 0 to 2526
Data columns (total 26 columns):
Staff ID                                                  2508 non-null int64
Change of career line                                     1080 non-null float64
Expectations mismatched                                   5 non-null float64
Family reasons                                            251 non-null float64
Further studies                                           517 non-null float64
Difficulty with job scope                                 72 non-null float64
Long working hours                                        714 non-null float64
Low basic pay                                             773 non-null float64
Personal reasons                                          112 non-null float64
Received a better job offer                               303 non-null float64
Left for higher salary                                    58 non-null float64
Retirement/taking a break   

Merge and write:

In [64]:
df3 = pd.merge(resigned, reasons_melted, how='left', on=['Staff ID'])
df3.drop(['value'], axis=1, inplace=True)
df3.fillna("Not specified", inplace=True)
df3.to_csv("resigned_clean.csv", index=False)